In [1]:
from pyspark.sql import SparkSession

import os
from pyspark.sql import SparkSession


# Initialize Spark session with HBase configuration
spark = SparkSession.builder \
    .appName("WriteToHbase") \
    .config("spark.jars", "/tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar,/tmp/hbase-spark-1.1.0-SNAPSHOT.jar") \
    .config("spark.extraListeners", "datahub.spark.DatahubSparkListener")\
    .config("spark.datahub.rest.server", "http://10.208.164.167:8080")\
    .config("spark.datahub.metadata.dataset.materialize", "true")\
    .config("spark.datahub.metadata.dataset.experimental_include_schema_metadata", "true")\
    .getOrCreate()

# .config("spark.hadoop.hbase.zookeeper.quorum", "localhost1") -> oke ăn config này nhé (còn đâu khả năng nó ăn mặc định, hoặc là file habse.xml tỏng classpath)
# .config("spark.hbase.host", "localhost:2182")  -> khong chay 
# print(os.environ["HBASE_HOME"])
# os.environ["HBASE_HOME"] ="/tmp/a" -> không work

spark.sparkContext.setLogLevel("DEBUG")

24/12/17 10:10:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
data = [('2','Telangana','Chhetri'),
  ('3','Sikkim','Bhutia'),
  ('4','Hyderabad','Shabbir'),
  ('5','Kerala','Vijayan'),
  ('6','Mizoram','Lalpekhlua')
  ]
columns = ["id","city","name"]
goalsDF = spark.createDataFrame(data=data, schema = columns)
goalsDF.show()

# hdfs 
# goalsDF.write.mode("overwrite").parquet("hdfs://hadoop:9000/tmp/test")
#spark.read.parquet("hdfs://hadoop:9000/tmp/test").show()


24/12/17 10:10:08 DEBUG FileSystem: Looking for FS supporting file
24/12/17 10:10:08 DEBUG FileSystem: looking for configuration option fs.file.impl
24/12/17 10:10:08 DEBUG FileSystem: Looking in service filesystems for implementation class
24/12/17 10:10:08 DEBUG FileSystem: FS for file is class org.apache.hadoop.hive.ql.io.ProxyLocalFileSystem
24/12/17 10:10:08 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/12/17 10:10:08 DEBUG SharedState: Applying other initial session options to HadoopConf: spark.datahub.metadata.dataset.experimental_include_schema_metadata -> true
24/12/17 10:10:08 DEBUG SharedState: Applying other initial session options to HadoopConf: spark.extraListeners -> datahub.spark.DatahubSparkListener
24/12/17 10:10:08 DEBUG SharedState: Applying other initial session options to HadoopConf: spark.jars -> /tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar,/tmp/hbase-spark-1.1.0-SNAPSHOT.jar
24/12/17 10:10:08 DEBUG S

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


24/12/17 10:10:10 DEBUG DatahubSparkListener: loadDatahubConfig completed successfully in 364 ms
24/12/17 10:10:10 INFO ArgumentParser: Couldn't log config from file, will read it from SparkConf
24/12/17 10:10:10 DEBUG InternalLoggerFactory: Using SLF4J as the default logging framework
24/12/17 10:10:10 DEBUG CompositeMeterRegistry: A MeterFilter is being configured after a Meter has been registered to this registry. All MeterFilters should be configured before any Meters are registered. If that is not possible or you have a use case where it should be allowed, let the Micrometer maintainers know at https://github.com/micrometer-metrics/micrometer/issues/4920.
java.lang.Thread.getStackTrace(Thread.java:1564)
	at io.micrometer.core.instrument.MeterRegistry$Config.logWarningAboutLateFilter(MeterRegistry.java:844)
	at io.micrometer.core.instrument.MeterRegistry$Config.meterFilter(MeterRegistry.java:830)
	at io.micrometer.core.instrument.MeterRegistry$Config.commonTags(MeterRegistry.java:8

24/12/17 10:10:11 INFO PythonRunner: Times: total = 325, boot = 278, init = 47, finish = 0
24/12/17 10:10:11 INFO Executor: Finished task 0.0 in stage 0.0 (TID 0). 1832 bytes result sent to driver
24/12/17 10:10:11 DEBUG ExecutorMetricsPoller: stageTCMP: (0, 0) -> 0
24/12/17 10:10:11 INFO TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 487 ms on hadoop113 (executor driver) (1/1)
24/12/17 10:10:11 INFO TaskSchedulerImpl: Removed TaskSet 0.0, whose tasks have all completed, from pool 
24/12/17 10:10:11 INFO PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 57781
24/12/17 10:10:11 DEBUG DatahubSparkListener: Task end called
24/12/17 10:10:11 DEBUG DatahubSparkListener: onTaskEnd completed successfully in 1 ms
24/12/17 10:10:11 INFO DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 0.595 s
24/12/17 10:10:11 DEBUG DAGScheduler: After removal of stage 0, remaining stages = 0
24/12/17 10:10:11 INFO DAGScheduler: Job 0 

In [3]:
#write to hbase

# hbase:001:0> create 'testspark', 'cf'
# Created table testspark
# Took 2.3158 seconds                                                                                                                                                                           
# => Hbase::Table - testspark
# hbase:003:0> scan 'testspark'

# sao k co ip hbase ma no van chay dc k nhi :D
# .option("hbase.zookeeper.quorum", "localhost:2182") \
goalsDF.write\
.format("org.apache.hadoop.hbase.spark")\
.option("hbase.columns.mapping","id STRING :key, name STRING cf:name, city STRING cf:city")\
.option("hbase.namespace", "default")\
.option("hbase.table", "testspark")\
.option("hbase.spark.use.hbasecontext", False)\
.save() 

24/12/17 10:10:11 DEBUG OpenLineageRunEventBuilder: Visiting query plan Optional[== Parsed Logical Plan ==
GlobalLimit 21
+- LocalLimit 21
   +- Project [cast(id#0 as string) AS id#9, cast(city#1 as string) AS city#10, cast(name#2 as string) AS name#11]
      +- LogicalRDD [id#0, city#1, name#2], false

== Analyzed Logical Plan ==
id: string, city: string, name: string
GlobalLimit 21
+- LocalLimit 21
   +- Project [cast(id#0 as string) AS id#9, cast(city#1 as string) AS city#10, cast(name#2 as string) AS name#11]
      +- LogicalRDD [id#0, city#1, name#2], false

== Optimized Logical Plan ==
GlobalLimit 21
+- LocalLimit 21
   +- LogicalRDD [id#0, city#1, name#2], false

== Physical Plan ==
CollectLimit 21
+- *(1) Scan ExistingRDD[id#0,city#1,name#2]
] with output dataset builders [<function1>, <function1>, <function1>, <function1>, <function1>, <function1>, <function1>, <function1>, <function1>, <function1>]
24/12/17 10:10:11 DEBUG SparkSQLExecutionContext: Posting event for end 0: {"e

[Stage 3:==========================================>              (15 + 5) / 20]

24/12/17 10:10:12 INFO ClientCnxn: EventThread shut down for session: 0x10004ff096301cd
24/12/17 10:10:12 INFO ClientCnxn: EventThread shut down for session: 0x10004ff096301ce
24/12/17 10:10:12 INFO ZooKeeper: Session: 0x10004ff096301cd closed
24/12/17 10:10:12 DEBUG ClientCnxn: Reading reply sessionid:0x10004ff096301d1, packet:: clientPath:/hbase serverPath:/hbase finished:false header:: 2,8  replyHeader:: 2,1605,0  request:: '/hbase,F  response:: v{'meta-region-server,'rs,'splitWAL,'backup-masters,'flush-table-proc,'master-maintenance,'online-snapshot,'switch,'master,'running,'draining,'namespace,'hbaseid,'table} 
24/12/17 10:10:12 DEBUG ClientCnxn: Reading reply sessionid:0x10004ff096301c2, packet:: clientPath:/hbase serverPath:/hbase finished:false header:: 2,8  replyHeader:: 2,1605,0  request:: '/hbase,F  response:: v{'meta-region-server,'rs,'splitWAL,'backup-masters,'flush-table-proc,'master-maintenance,'online-snapshot,'switch,'master,'running,'draining,'namespace,'hbaseid,'tabl

24/12/17 10:10:13 DEBUG OpenLineageRunEventBuilder: Physical plan executed [{"class":"org.apache.spark.sql.execution.command.ExecutedCommandExec","num-children":0,"cmd":[{"class":"org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand","num-children":0,"query":[{"class":"org.apache.spark.sql.execution.LogicalRDD","num-children":0,"output":[[{"class":"org.apache.spark.sql.catalyst.expressions.AttributeReference","num-children":0,"name":"id","dataType":"string","nullable":true,"metadata":{},"exprId":{"product-class":"org.apache.spark.sql.catalyst.expressions.ExprId","id":0,"jvmId":"5c8e9f13-2997-4228-9187-09547e0dd1aa"},"qualifier":[]}],[{"class":"org.apache.spark.sql.catalyst.expressions.AttributeReference","num-children":0,"name":"city","dataType":"string","nullable":true,"metadata":{},"exprId":{"product-class":"org.apache.spark.sql.catalyst.expressions.ExprId","id":1,"jvmId":"5c8e9f13-2997-4228-9187-09547e0dd1aa"},"qualifier":[]}],[{"class":"org.apache.spark.sql.catalyst